In [1]:
import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import math
from tensorboardX import SummaryWriter

import import_ipynb
from CustomDataset import ControlsDataset
from Model import ConvNet

REPORT_EVERY_ITER = 100

importing Jupyter notebook from CustomDataset.ipynb
importing Jupyter notebook from Model.ipynb


In [2]:
dataset = ControlsDataset()
dataloader = DataLoader(dataset, batch_size = 16, shuffle = True, num_workers = 16)

net = ConvNet()
print("number of parameters: ", sum(p.numel() for p in net.parameters()))

units after conv 768
number of parameters:  1111969


In [6]:
epochs = 100
optimizer = optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.MSELoss()
writer = SummaryWriter()
inter_no = 0

# for i_batch, sampled_batch in enumerate(dataloader):
#     for epoch in range(epochs):
for epoch in range(epochs):
    for i_batch, sampled_batch in enumerate(dataloader):
        inter_no += 1
        images = sampled_batch['image'].float()
        controls = sampled_batch['control'].float()
    
        optimizer.zero_grad()
        prediction = net(images)
        
        loss = criterion(prediction, controls)
        loss.backward()
        optimizer.step()

        if inter_no % REPORT_EVERY_ITER == 0:
            writer.add_scalar("Loss", loss.item(), inter_no)
        
        print(epoch, loss.data)


0 tensor(25248.6992)
1 tensor(25210.4023)
2 tensor(25175.0449)
3 tensor(25128.2598)
4 tensor(25061.4395)
5 tensor(24963.9219)
6 tensor(24830.5098)
7 tensor(24642.6953)
8 tensor(24384.8945)
9 tensor(24029.7754)
10 tensor(23544.4395)
11 tensor(22908.0879)
12 tensor(22064.6582)
13 tensor(20961.4863)
14 tensor(19563.3711)
15 tensor(17809.9023)
16 tensor(15671.4150)
17 tensor(13230.2217)
18 tensor(10650.3730)
19 tensor(8380.2734)
20 tensor(7167.8750)
21 tensor(7605.4595)
22 tensor(9400.8721)
23 tensor(8598.7705)
24 tensor(6738.5557)
25 tensor(5804.5796)
26 tensor(5707.9932)
27 tensor(5100.5977)
28 tensor(5568.7266)
29 tensor(5420.1055)
30 tensor(4775.6333)
31 tensor(4671.5933)
32 tensor(4178.5757)
33 tensor(3535.5825)
34 tensor(3571.3821)
35 tensor(3185.0767)
36 tensor(3175.0681)
37 tensor(3104.3049)
38 tensor(2815.5095)
39 tensor(2743.8958)
40 tensor(2358.3992)
41 tensor(2443.5322)
42 tensor(2266.7036)
43 tensor(2238.3901)
44 tensor(2102.9910)
45 tensor(1849.9423)
46 tensor(1828.3541)
47 t

KeyboardInterrupt: 